In [1]:
import sqlite3
import random
from datetime import datetime
import string

# Connect to SQLite (this will create a database file in your working directory)
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

In [2]:
cursor.execute('SELECT sqlite_version();')
print(cursor.fetchone())  # Print the SQLite version

('3.45.3',)


In [3]:
cursor.execute('''SELECT name FROM sqlite_master WHERE type='table';''')
tables = cursor.fetchall()
print(tables)

[]


In [4]:
# Close the connection
conn.close()

In [45]:
# Note: Dont run this cell unless you require it to do 

# Connect to the SQLite database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# Drop the Users table
cursor.execute('DROP TABLE IF EXISTS x')

# Commit the transaction to apply the changes
conn.commit()

# Confirm the table has been deleted
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Remaining tables:", tables)

# Close the connection
conn.close()

Remaining tables: [('Users',), ('Instructors',), ('sqlite_sequence',), ('Courses',), ('Offline_Branches',), ('Enrollments',), ('Reviews',)]


In [5]:
import sqlite3
import random
from faker import Faker

# Connect to SQLite database (creates 'hybrid_learning1.db' if it doesn't exist)
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# Create the Users table with necessary constraints
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT NOT NULL,
        age INTEGER,
        subscription_level TEXT CHECK (subscription_level IN ('Basic', 'Premium', 'VIP')),
        registration_date DATE NOT NULL,
        mode_of_learning TEXT CHECK (mode_of_learning IN ('Online', 'Offline')),
        contact_number TEXT,
        address TEXT,
        gender TEXT CHECK (gender IN ('Male', 'Female', 'Other'))
    )
''')
--conn.commit()

# Initialize Faker instance to generate fake data
fake = Faker()

# Function to generate realistic user data
def generate_user_data(num_records):
    user_data = []
    duplicate_names = set()  # This can be removed for true duplicates

    for _ in range(num_records):
        # Generate realistic data for critical columns
        name = fake.name()  # Allow duplicates naturally by not tracking unique names
        email = fake.unique.email()  # Unique email to avoid duplicates
        age = random.randint(18, 70)
        subscription_level = random.choice(['Basic', 'Premium', 'VIP'])
        registration_date = fake.date_this_decade()
        mode_of_learning = random.choice(['Online', 'Offline'])
        gender = random.choice(['Male', 'Female', 'Other'])
        
        # Optional fields, where missing data is simulated
        contact_number = fake.phone_number() if random.random() < 0.8 else None  
                                        # 80% chance of having a contact number
        address = fake.address() if random.random() < 0.8 else None  
                                        # 80% chance of having an address
        
        # Add user data to the list
        user_data.append((name, email, age, subscription_level,registration_date,
                          mode_of_learning, contact_number, address, gender))
    
    return user_data

# Generate 1300 records (or adjust as per requirement)
user_data = generate_user_data(1300)



# Insert data into the Users table (using batches for efficiency)
batch_size = 500  # Adjust batch size for performance
for i in range(0, len(user_data), batch_size):
    batch = user_data[i:i + batch_size]
    try:
        cursor.executemany('''
            INSERT INTO Users (name, email, age, subscription_level, registration_date, mode_of_learning, contact_number, address, gender)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', batch)
        conn.commit()  # Commit after each batch
    except sqlite3.OperationalError as e:
        print(f"Error inserting batch {i}-{i+batch_size}: {e}")
        
cursor.execute('SELECT * FROM Users LIMIT 10')  # Fetch a few records to inspect
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the database connection
conn.close()

# Output to check if data was inserted
print(f'{len(user_data)} records inserted into the Users table.')





(1, 'Roger Ritter', 'cliffordgoodman@example.net', 30, 'VIP', '2024-04-10', 'Online', '4425911236', '5745 Simon Park\nSouth Brianmouth, AZ 39786', 'Male')
(2, 'Dawn Li', 'tamara10@example.com', 69, 'Premium', '2024-06-20', 'Online', '244.434.4982x008', '69869 Brenda Wall Suite 610\nMeganbury, RI 99428', 'Female')
(3, 'Vincent Campbell', 'fbrooks@example.com', 65, 'VIP', '2022-05-30', 'Offline', '975-896-9445x287', '3713 Sarah Path Suite 708\nEast Jerry, CA 89076', 'Other')
(4, 'Nathan Spence', 'dgates@example.com', 51, 'Basic', '2021-06-24', 'Offline', '678.622.6297x53379', 'PSC 9830, Box 6798\nAPO AA 21282', 'Other')
(5, 'Robert Rios Jr.', 'ksaunders@example.com', 41, 'Basic', '2024-03-23', 'Offline', None, 'USS Sawyer\nFPO AE 73080', 'Male')
(6, 'Amanda Barber', 'rachelthompson@example.net', 43, 'Basic', '2020-01-31', 'Online', '332-415-8123x784', '044 Peter Dale Apt. 318\nEast Teresafurt, MS 46391', 'Other')
(7, 'Ryan Harris', 'jamesbrown@example.net', 64, 'Basic', '2024-06-11', 'On

In [6]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# 1. Check for NULL values in critical columns
print("1. Checking for NULL values in critical columns...")
cursor.execute('''
    SELECT COUNT(*) 
    FROM Users 
    WHERE name IS NULL OR email IS NULL OR subscription_level IS NULL OR registration_date IS NULL OR mode_of_learning IS NULL OR gender IS NULL
''')
null_values = cursor.fetchone()[0]
print(f"Number of records with NULL values in critical columns: {null_values}")

# 2. Check for constraint violations
print("\n2. Checking for constraint violations...")
cursor.execute('''
    SELECT COUNT(*) 
    FROM Users 
    WHERE subscription_level NOT IN ('Basic', 'Premium', 'VIP') OR 
          mode_of_learning NOT IN ('Online', 'Offline') OR 
          gender NOT IN ('Male', 'Female', 'Other')
''')
constraint_violations = cursor.fetchone()[0]
print(f"Number of records violating constraints: {constraint_violations}")

# 3. Check for missing and duplicate data
print("\n3. Checking for missing and duplicate data...")

# Missing data
cursor.execute('''
    SELECT COUNT(*) 
    FROM Users 
    WHERE age IS NULL OR contact_number IS NULL OR address IS NULL
''')
missing_data = cursor.fetchone()[0]
print(f"Number of records with missing optional data: {missing_data}")

# Duplicate names
cursor.execute('''
    SELECT name, COUNT(*) 
    FROM Users 
    GROUP BY name 
    HAVING COUNT(*) > 1
''')
duplicate_names = cursor.fetchall()
print(f"Number of duplicate names: {len(duplicate_names)}")
if len(duplicate_names) > 0:
    print("Examples of duplicate names:")
    print(duplicate_names[:5])  # Show a few duplicates

# 4. Check total rows
print("\n4. Checking total rows...")
cursor.execute('SELECT COUNT(*) FROM Users')
total_rows = cursor.fetchone()[0]
print(f"Total rows in the Users table: {total_rows}")

# Close the connection
conn.close()


1. Checking for NULL values in critical columns...
Number of records with NULL values in critical columns: 0

2. Checking for constraint violations...
Number of records violating constraints: 0

3. Checking for missing and duplicate data...
Number of records with missing optional data: 453
Number of duplicate names: 11
Examples of duplicate names:
[('Alyssa Johnson', 2), ('Christopher Spencer', 2), ('Daniel Young', 2), ('Larry Herrera', 2), ('Mary Johnson', 2)]

4. Checking total rows...
Total rows in the Users table: 1300


In [7]:
import random
from faker import Faker
import sqlite3

# Initialize Faker instance to generate random data
fake = Faker()

# Create a connection to your SQLite database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()


# Creating the Instructors table with appropriate columns and constraints
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Instructors (
        instructor_id INTEGER PRIMARY KEY AUTOINCREMENT,
        instructor_name TEXT NOT NULL,
        specialization TEXT,
        hire_date TEXT NOT NULL CHECK(hire_date LIKE '____-__-__'),  -- Ensure date format YYYY-MM-DD
        mode_of_learning TEXT NOT NULL CHECK(mode_of_learning IN ('Online', 'Offline')),  -- Ensure valid mode
        contact_num TEXT,
        email TEXT NOT NULL UNIQUE  -- Ensure email is unique
    )
''')

# Commit the transaction to create the table
conn.commit()



# Function to generate unique emails
def generate_email(specialization, existing_emails):
    while True:
        # Generate a realistic email based on the specialization
        email = fake.first_name().lower() + '.' + specialization.lower() + '@example.com'
        
        # Check if email already exists, if not add it to the set and return it
        if email not in existing_emails:
            existing_emails.add(email)
            return email
        
        # In case of a conflict, create a variation like adding a number
        email = fake.first_name().lower() + '.' + specialization.lower() + str(random.randint(1, 100)) + '@example.com'
        
        # If the email is not in use, add it and return
        if email not in existing_emails:
            existing_emails.add(email)
            return email

# Set to track used emails
used_emails = set()

# Insert 100 instructors with missing data and duplicates
for _ in range(100):  # 100 instructors
    # Simulating realistic missing data and duplicate values

    # Name (10% chance of being missing, allowing duplicates)
    instructor_name = fake.name() if random.random() > 0.1 else "Unknown"
    
    # Specialization (5% chance of being missing, allowing duplicates)
    specialization = random.choice(['Mathematics', 'Biology', 'Physics', 'Chemistry', 'Zoology']) if random.random() > 0.05 else "Unknown"
    
    # Hire Date (more recent dates, 20% chance of being missing, default to "2020-01-01")
    hire_date = fake.date_this_decade() if random.random() > 0.2 else "2020-01-01"
    
    # Mode of Learning (5% chance of being missing, allowing duplicates)
    mode_of_learning = random.choice(['Online', 'Offline']) if random.random() > 0.05 else "Online"
    
    # Contact Number (15% chance of being missing, allowing duplicates)
    contact_number = fake.phone_number() if random.random() > 0.15 else "Unknown"
    
    # Email (generated with the function to avoid duplicates)
    email = generate_email(specialization, used_emails)
    
    # Creating instructor data tuple
    instructor_data = (
        instructor_name,
        specialization,
        hire_date,
        mode_of_learning,
        contact_number,
        email
    )
    
    try:
        cursor.execute(''' 
            INSERT INTO Instructors (instructor_name, specialization, hire_date, mode_of_learning, contact_num, email)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', instructor_data)
    except sqlite3.IntegrityError as e:
        print("Error inserting instructor data:", e)

# Commit the transaction to save changes
conn.commit()

# Check how many rows were inserted
cursor.execute('SELECT COUNT(*) FROM Instructors')
row_count = cursor.fetchone()[0]
print(f'Total rows in Instructors table: {row_count}')


cursor.execute('SELECT * FROM Instructors LIMIT 10')  # Fetch a few records to inspect
rows = cursor.fetchall()
for row in rows:
    print(row)
# Close the connection
conn.close()


Total rows in Instructors table: 100
(1, 'Anita Gonzalez', 'Biology', '2024-10-06', 'Online', '(539)374-9092x332', 'cameron.biology@example.com')
(2, 'Christopher Collins', 'Physics', '2020-01-01', 'Online', '594-463-5328', 'theresa.physics@example.com')
(3, 'Sandra Anderson', 'Physics', '2020-01-01', 'Online', '+1-363-635-1725', 'jessica.physics@example.com')
(4, 'Christina Singh', 'Biology', '2022-03-28', 'Online', 'Unknown', 'paul.biology@example.com')
(5, 'Unknown', 'Zoology', '2024-03-21', 'Offline', '(541)865-6492x91923', 'ashley.zoology@example.com')
(6, 'Tara Anderson', 'Mathematics', '2022-02-21', 'Online', 'Unknown', 'maria.mathematics@example.com')
(7, 'Ryan Owens', 'Physics', '2020-03-15', 'Offline', '5932875502', 'catherine.physics@example.com')
(8, 'Kari Lam', 'Physics', '2023-10-11', 'Offline', '9498397084', 'aaron.physics@example.com')
(9, 'Leah Mcdonald', 'Physics', '2023-01-06', 'Online', '952.887.0224x42811', 'yolanda.physics@example.com')
(10, 'Unknown', 'Biology', 

In [8]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# Check for NULL values in critical columns
cursor.execute('''
    SELECT COUNT(*) 
    FROM Instructors 
    WHERE instructor_name IS NULL OR hire_date IS NULL OR email IS NULL
''')
null_values = cursor.fetchone()[0]
print(f'Number of records with NULL values in critical columns: {null_values}')

# Check for duplicate emails
cursor.execute('''
    SELECT email, COUNT(*)
    FROM Instructors
    GROUP BY email
    HAVING COUNT(*) > 1
''')
duplicate_emails = cursor.fetchall()
print(f'Duplicate emails: {duplicate_emails}')


# Check for duplicate contact numbers
cursor.execute('''
    SELECT contact_num, COUNT(*) 
    FROM Instructors
    GROUP BY contact_num
    HAVING COUNT(*) > 1;
''')
duplicate_contact_numbers = cursor.fetchall()
print("Duplicate Contact Numbers:", duplicate_contact_numbers)

# Check for duplicate specializations
cursor.execute('''
    SELECT specialization, COUNT(*) 
    FROM Instructors
    GROUP BY specialization
    HAVING COUNT(*) > 1;
''')
duplicate_specializations = cursor.fetchall()
print("Duplicate Specializations:", duplicate_specializations)

# Check for duplicate modes of learning
cursor.execute('''
    SELECT mode_of_learning, COUNT(*) 
    FROM Instructors
    GROUP BY mode_of_learning
    HAVING COUNT(*) > 1;
''')
duplicate_modes = cursor.fetchall()
print("Duplicate Modes of Learning:", duplicate_modes)


# Check for missing or unrealistic entries
cursor.execute('''
    SELECT COUNT(*) 
    FROM Instructors 
    WHERE instructor_name = "Unknown" OR specialization = "Unknown" OR contact_num = "Unknown"
''')
missing_or_unrealistic = cursor.fetchone()[0]
print(f'Number of records with missing or unrealistic values: {missing_or_unrealistic}')

# Check for total rows in the table
cursor.execute('SELECT COUNT(*) FROM Instructors')
total_rows = cursor.fetchone()[0]
print(f'Total rows in Instructors table: {total_rows}')

# Close the connection
conn.close()


Number of records with NULL values in critical columns: 0
Duplicate emails: []
Duplicate Contact Numbers: [('Unknown', 11)]
Duplicate Specializations: [('Biology', 15), ('Chemistry', 19), ('Mathematics', 14), ('Physics', 18), ('Unknown', 6), ('Zoology', 28)]
Duplicate Modes of Learning: [('Offline', 45), ('Online', 55)]
Number of records with missing or unrealistic values: 26
Total rows in Instructors table: 100


In [9]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to SQLite database (or create if it doesn't exist)
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()


# Step 1: Create the Courses table with constraints and foreign key handling
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT NOT NULL,
        specialization TEXT NOT NULL,
        instructor_id INTEGER NOT NULL,
        mode_of_learning TEXT CHECK (mode_of_learning IN ('Online', 'Offline')),
        price INTEGER,  -- Some courses may not have a price (null)
        duration INTEGER,  -- Some courses may not have a defined duration (null)
        launch_date DATE NOT NULL,
        course_description TEXT,  -- Some courses may not have a description (null)
        prerequisites TEXT,  -- Some courses may not have prerequisites (null)
        FOREIGN KEY (instructor_id) REFERENCES Instructors(instructor_id) ON DELETE RESTRICT
    )
''')
conn.commit()
# Step 2: Define random data for course creation
course_names = ['Introduction to Algebra', 'Quantum Mechanics', 'Organic Chemistry','Biology for Beginners', 'Advanced Linear Algebra',
                 'Mechanics', 'Data Science Fundamentals', 'Advanced Python', 'Machine Learning Basics']
specializations = ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'Zoology']
modes_of_learning = ['Online', 'Offline']
descriptions = ['Learn the basics of this subject', 'Deep dive into advanced topics', 'Introduction to foundational concepts', 
                          'In-depth understanding of subject matter', None]  # None represents missing descriptions
prerequisites = ['Basic Mathematics', 'Basic Physics', 'Basic Chemistry', 'None', None]  # None represents missing prerequisites

# Generate random dates
def random_date(start_date, end_date):
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)

# Step 3: Generate 100 random courses with realistic missing/duplicate data
course_data = []
unique_course_names = set(course_names)  # Ensuring unique course names
unique_specializations = set(specializations)  # Ensuring unique specializations
unique_modes = set(modes_of_learning)  # Ensuring unique modes of learning
for i in range(1, 101):  # Generate 100 courses
    course_name = random.choice(list(unique_course_names)) if random.random() > 0.07 else "Unknown Course"
    specialization = random.choice(list(unique_specializations)) if random.random() > 0.12 else "Unknown Specialization"
    mode_of_learning = random.choice(list(unique_modes)) if random.random() > 0.08 else "Offline"
    
    # Randomly generate other data with missing values
    price = random.randint(100, 500) if random.random() > 0.2 else None
    duration = random.choice([20, 30, 40, 50]) if random.random() > 0.3 else None
    launch_date = random_date(start_date, end_date).strftime('%Y-%m-%d')
    course_description = random.choice(descriptions)
    prerequisite = random.choice(prerequisites)
    
    # Assign instructor_id (assuming 100 instructors exist)
    instructor_id = random.randint(1, 100)

    # Append generated course data
    course_data.append((
        i, 
        course_name, 
        specialization, 
        instructor_id, 
        mode_of_learning, 
        price, 
        duration, 
        launch_date, 
        course_description, 
        prerequisite
    ))
# Step 4: Insert the data into the Courses table (with missing and duplicate values)
cursor.executemany('''
    INSERT INTO Courses (course_id, course_name, specialization, instructor_id, mode_of_learning, price, duration, launch_date, course_description, prerequisites)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', course_data)

# Commit the changes
conn.commit()

# Display confirmation message
print("Courses table created and randomized data inserted!")

# Step 5: Fetch and display a few rows from the Courses table
cursor.execute("SELECT * FROM Courses LIMIT 10")
rows = cursor.fetchall()

# Display the fetched rows
for row in rows:
    print(row)

# Close the connection
conn.close()


Courses table created and randomized data inserted!
(1, 'Data Science Fundamentals', 'Biology', 11, 'Online', 381, 50, '2024-09-23', 'In-depth understanding of subject matter', 'Basic Physics')
(2, 'Mechanics', 'Biology', 96, 'Online', 107, 20, '2023-07-04', 'Deep dive into advanced topics', 'Basic Chemistry')
(3, 'Data Science Fundamentals', 'Zoology', 46, 'Online', 262, 50, '2024-03-27', 'Deep dive into advanced topics', 'Basic Chemistry')
(4, 'Biology for Beginners', 'Chemistry', 37, 'Offline', 360, 20, '2023-02-05', None, None)
(5, 'Organic Chemistry', 'Unknown Specialization', 43, 'Online', None, 50, '2023-01-11', 'In-depth understanding of subject matter', 'Basic Physics')
(6, 'Quantum Mechanics', 'Unknown Specialization', 83, 'Online', 176, 50, '2024-04-19', 'Learn the basics of this subject', 'Basic Physics')
(7, 'Introduction to Algebra', 'Unknown Specialization', 38, 'Online', 460, 40, '2023-12-02', 'Deep dive into advanced topics', 'Basic Chemistry')
(8, 'Biology for Beginne

In [10]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# 1. Check for missing values in critical and non-critical columns
cursor.execute("SELECT COUNT(*) FROM Courses WHERE course_name IS NULL")
missing_course_names = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM Courses WHERE specialization IS NULL")
missing_specializations = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM Courses WHERE mode_of_learning IS NULL")
missing_modes_of_learning = cursor.fetchone()[0]

# 2. Check for duplicates in non-critical columns (course_name, specialization, mode_of_learning)
cursor.execute("SELECT course_name, COUNT(*) FROM Courses GROUP BY course_name HAVING COUNT(*) > 1")
duplicate_course_names = cursor.fetchall()

cursor.execute("SELECT specialization, COUNT(*) FROM Courses GROUP BY specialization HAVING COUNT(*) > 1")
duplicate_specializations = cursor.fetchall()

cursor.execute("SELECT mode_of_learning, COUNT(*) FROM Courses GROUP BY mode_of_learning HAVING COUNT(*) > 1")
duplicate_modes_of_learning = cursor.fetchall()

# 3. Ensure foreign key integrity (no invalid instructor_id)
cursor.execute("SELECT COUNT(*) FROM Courses WHERE instructor_id NOT IN (SELECT instructor_id FROM Instructors)")
invalid_instructor_ids = cursor.fetchone()[0]

# Display results
print(f"Missing course names: {missing_course_names}")
print(f"Missing specializations: {missing_specializations}")
print(f"Missing modes of learning: {missing_modes_of_learning}")
print(f"Duplicate course names: {duplicate_course_names}")
print(f"Duplicate specializations: {duplicate_specializations}")
print(f"Duplicate modes of learning: {duplicate_modes_of_learning}")
print(f"Invalid instructor_id (foreign key constraint check): {invalid_instructor_ids}")

# Close the connection
conn.close()

Missing course names: 0
Missing specializations: 0
Missing modes of learning: 0
Duplicate course names: [('Advanced Linear Algebra', 9), ('Advanced Python', 8), ('Biology for Beginners', 16), ('Data Science Fundamentals', 11), ('Introduction to Algebra', 13), ('Machine Learning Basics', 14), ('Mechanics', 10), ('Organic Chemistry', 6), ('Quantum Mechanics', 10), ('Unknown Course', 3)]
Duplicate specializations: [('Biology', 15), ('Chemistry', 27), ('Mathematics', 11), ('Physics', 16), ('Unknown Specialization', 11), ('Zoology', 20)]
Duplicate modes of learning: [('Offline', 52), ('Online', 48)]
Invalid instructor_id (foreign key constraint check): 0


In [11]:
import sqlite3
from faker import Faker
import random

# Initialize Faker instance
fake = Faker()

# Randomized data for the Offline_Branches table
branch_data = []
facility_details_list = [
    "Wi-Fi, Projectors, Smartboards, Air-conditioning", 
    "Wi-Fi, Whiteboards, Computers, Air-conditioning", 
    "Wi-Fi, Projectors, Smartboards, Parking space", 
    "Wi-Fi, Whiteboards, Advanced laboratory equipment", 
    "Wi-Fi, Virtual Reality setup, Parking space"
]

# Function to generate random class schedule
def generate_random_schedule():
    days_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    num_days = random.randint(3, 7)  # Branches could be open between 3 and 7 days a week
    working_days = random.sample(days_of_week, num_days)  # Randomly choose days
    working_days.sort()  # Sort the days alphabetically for consistency
    start_hour = random.randint(8, 11)  # Start time between 8 AM to 11 AM
    end_hour = random.randint(4, 7)  # End time between 4 PM to 7 PM
    schedule = f"{', '.join(working_days)}, {start_hour}AM - {end_hour}PM"
    return schedule

# Function to generate random facility details
def generate_random_facilities():
    return random.choice(facility_details_list)

# Function to create and insert data into the Offline_Branches table
def create_and_insert_branches():
    # Open the database connection
    conn = sqlite3.connect('hybrid_learning@in.db')
    cursor = conn.cursor()

    # Create the Offline_Branches table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Offline_Branches (
        branch_id INTEGER PRIMARY KEY,
        branch_name TEXT NOT NULL,
        location TEXT NOT NULL,
        contact_info TEXT,
        class_schedule TEXT,
        instructor_id INTEGER,
        max_students INTEGER NOT NULL,
        facility_details TEXT,
        FOREIGN KEY (instructor_id) REFERENCES Instructors (instructor_id)
    )
    ''')

    # Generate 62 random branches
    for i in range(1,62 ):
        branch_name = fake.city() + " Branch"  # Corrected method to generate random branch name
        location = fake.city()  # Generate random location name
        contact_info = fake.phone_number()  # Generate random phone number
        class_schedule = generate_random_schedule()  # Generate random class schedule
        instructor_id = random.randint(1, 100)  # Random instructor ID (assuming 100 instructors exist)
        max_students = random.randint(30, 100)  # Random max number of students between 30 and 100
        facility_detail = generate_random_facilities()  # Generate random facility details

        # Randomly introduce missing values for realism (10% chance for missing values)
        if random.random() < 0.1:
            contact_info = None
        if random.random() < 0.1:
            class_schedule = None
        if random.random() < 0.1:
            facility_detail = None

        branch_data.append((
            i,  # branch_id (just an incremental ID for now)
            branch_name,
            location,
            contact_info,
            class_schedule,
            instructor_id,
            max_students,
            facility_detail
        ))

    # Insert the randomized data into the Offline_Branches table
    cursor.executemany('''
        INSERT INTO Offline_Branches (branch_id, branch_name, location, contact_info, class_schedule, instructor_id, max_students, facility_details)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', branch_data)

    # Commit the changes
    conn.commit()

    # Fetch the first 10 rows from the Offline_Branches table
    cursor.execute('''SELECT * FROM Offline_Branches LIMIT 10''')

    # Fetch the data and print it
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    # Close the connection after all operations are completed
    conn.close()

    print("Offline_Branches table created and randomized data inserted with random class schedules and facility details!")

# Run the function
create_and_insert_branches()



(1, 'East Alex Branch', 'Matthewborough', '229.204.8169', 'Fri, Sun, Thu, Tue, Wed, 10AM - 4PM', 7, 32, 'Wi-Fi, Projectors, Smartboards, Parking space')
(2, 'Lorimouth Branch', 'Clementsberg', '860-403-5830x13096', 'Fri, Tue, Wed, 10AM - 5PM', 8, 40, 'Wi-Fi, Virtual Reality setup, Parking space')
(3, 'Barnesview Branch', 'Donaldport', '+1-802-397-1420x0146', 'Fri, Sat, Sun, Thu, Tue, Wed, 10AM - 5PM', 36, 97, 'Wi-Fi, Whiteboards, Computers, Air-conditioning')
(4, 'Clarkberg Branch', 'South Thomas', '404.933.9009', 'Fri, Sat, Sun, Thu, Tue, 11AM - 5PM', 6, 88, 'Wi-Fi, Whiteboards, Advanced laboratory equipment')
(5, 'Stephanieview Branch', 'West Michael', '+1-543-562-0302', 'Fri, Mon, Sun, Tue, Wed, 8AM - 4PM', 29, 98, 'Wi-Fi, Projectors, Smartboards, Air-conditioning')
(6, 'Espinozachester Branch', 'East Christopherborough', None, 'Mon, Sat, Tue, Wed, 10AM - 7PM', 43, 51, 'Wi-Fi, Virtual Reality setup, Parking space')
(7, 'Dyerview Branch', 'North Monique', '391.843.5805x9687', 'Fri, M

In [12]:
# Open the database connection
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()
# Check for invalid foreign keys (instructor_id) in Offline_Branches
cursor.execute('''
    SELECT b.branch_id, b.branch_name, b.instructor_id
    FROM Offline_Branches b
    LEFT JOIN Instructors i ON b.instructor_id = i.instructor_id
    WHERE i.instructor_id IS NULL
''')

invalid_instructors = cursor.fetchall()

if invalid_instructors:
    print("Invalid foreign keys (instructor_id) found:")
    for row in invalid_instructors:
        print(row)
else:
    print("All instructor_ids are valid.")
# Close the connection 
conn.close()


All instructor_ids are valid.


In [33]:
import sqlite3
import random
from faker import Faker
import datetime

fake = Faker()

# Connect to the database
with sqlite3.connect('hybrid_learning@in.db') as conn:
    cursor = conn.cursor()

    # Create the Enrollments table if it doesn't exist
    cursor.execute(''' 
    CREATE TABLE IF NOT EXISTS Enrollments (
        enrollment_id INTEGER PRIMARY KEY,
        user_id INTEGER,
        course_id INTEGER,
        enrollment_date TEXT,
        branch_id INTEGER,
        payment_status TEXT,
        completion_status TEXT,
        grade TEXT,
        enrollment_type TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id),
        FOREIGN KEY (course_id) REFERENCES Courses(course_id),
        FOREIGN KEY (branch_id) REFERENCES Offline_Branches(branch_id),
        UNIQUE(user_id, course_id, enrollment_type)  -- Composite unique constraint
    )
    ''')

    # Generate random data for Enrollments with missing and duplicate data
    enrollment_data = []

    # Define possible values
    payment_statuses = ['Paid', 'Pending', None]  # Added None for missing data
    completion_statuses = ['Completed', 'In Progress', None]  # Added None for missing data
    grades = ['A', 'B', 'C', 'D', 'F', None]  # Added None for missing data

    # Generate 1000 random enrollments with missing and duplicate data
    for i in range(1, 1001):
        user_id = random.randint(1, 100)  # Random user from Users table
        course_id = random.randint(1, 100)  # Random course from Courses table
        enrollment_date = fake.date_this_decade() if random.random() > 0.1 else None  # 10% chance of missing enrollment date

        # Get course mode (offline/online) from Courses table
        cursor.execute('SELECT mode_of_learning FROM Courses WHERE course_id = ?', (course_id,))
        course_mode = cursor.fetchone()

        if course_mode and course_mode[0] == 'Offline':
            branch_id = random.randint(1, 100)  # Random branch from Offline_Branches
        else:
            branch_id = None  # No branch for online courses
        
        payment_status = random.choice(payment_statuses)  # Randomly assign 'Paid', 'Pending', or None
        completion_status = random.choice(completion_statuses)  # Randomly assign 'Completed', 'In Progress', or None
        grade = random.choice(grades)  # Randomly assign grades or None
        enrollment_type = 'Offline' if course_mode and course_mode[0] == 'Offline' else 'Online'

        # Add missing data (e.g., % of records will have a NULL value for payment_status, completion_status, or grade)
        if random.random() < 0.05:
            payment_status = None  # 10% chance of missing payment status
        if random.random() < 0.1:
            completion_status = None  # 6% chance of missing completion status
        if random.random() < 0.06:
            grade = None  # 6% chance of missing grade

        # Add duplicate data intentionally (e.g., the same user enrolling in the same course twice)
        if i % 100 == 0:  # Duplicate every 100th record
            enrollment_data.append(enrollment_data[i % 100])  # Duplicate previous enrollment entry

        # Add intentional duplicates for payment_status, completion_status, and grade
        if i % 5 == 0:  # Duplicate every 5th record for payment_status
            payment_status = 'Paid'
        if i % 10 == 0:  # Duplicate every 10th record for completion_status
            completion_status = 'Completed'
        if i % 15 == 0:  # Duplicate every 15th record for grade
            grade = 'A'

        # Add record to enrollment data list
        enrollment_data.append((
            None,  # We will use None here to let SQLite auto-increment the enrollment_id
            user_id,
            course_id,
            enrollment_date,
            branch_id,
            payment_status,
            completion_status,
            grade,
            enrollment_type
        ))
        # Insert data into Enrollments table, ignoring duplicates
        cursor.executemany('''
        INSERT OR IGNORE INTO Enrollments (
        enrollment_id, user_id, course_id, enrollment_date, branch_id,
        payment_status, completion_status, grade, enrollment_type
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', enrollment_data)


    # Commit the changes (done automatically with 'with' block)
    print("Enrollments table created and randomized data inserted with missing and duplicate values!")                                                                 

    # Verify data inserted
    cursor.execute("SELECT * FROM Enrollments LIMIT 10")
    print(cursor.fetchall())

    # Commit the changes
    conn.commit()
    
# Close the connection
conn.close()


Enrollments table created and randomized data inserted with missing and duplicate values!
[(1, 78, 3, '2024-02-18', None, None, 'In Progress', None, 'Online'), (2, 82, 59, '2021-02-18', 92, None, 'Completed', None, 'Offline'), (3, 6, 42, '2020-09-19', 33, 'Paid', 'Completed', 'F', 'Offline'), (4, 34, 17, '2022-01-24', None, None, 'Completed', 'F', 'Online'), (5, 27, 40, '2023-03-27', 99, None, None, None, 'Offline'), (6, 77, 25, '2024-09-01', 55, None, 'Completed', None, 'Offline'), (7, 52, 66, '2022-10-03', 36, None, None, 'B', 'Offline'), (8, 19, 53, '2024-08-14', None, None, 'In Progress', None, 'Online'), (9, 42, 60, None, 87, 'Pending', None, None, 'Offline'), (10, 49, 65, '2020-09-02', None, 'Paid', 'Completed', 'B', 'Online')]


In [35]:
# Connect to the database
with sqlite3.connect('hybrid_learning@in.db') as conn:
    cursor = conn.cursor()

# Test 1: Check for duplicate enrollments (user_id, course_id, and enrollment_type)
    cursor.execute('''
        SELECT user_id, course_id, enrollment_type, COUNT(*)
        FROM Enrollments
        GROUP BY user_id, course_id, enrollment_type
        HAVING COUNT(*) > 1
    ''')
    duplicates = cursor.fetchall()

    if duplicates:
        print("Duplicate enrollments found:")
        for duplicate in duplicates:
            print(duplicate)
    else:
        print("No duplicate enrollments.")

    # Query to count duplicates in payment_status, completion_status, and grade together
    cursor.execute('''
    SELECT COUNT(*)
    FROM (
        SELECT payment_status, completion_status, grade
        FROM Enrollments
        GROUP BY payment_status, completion_status, grade
        HAVING COUNT(*) > 1
    ) AS duplicate_group
    ''')
    duplicate_count = cursor.fetchone()[0]  # Fetch the count from the query result
    print("Number of duplicate groups (based on payment_status, completion_status, and grade):", duplicate_count)


    # Test 2: Check for missing values in payment_status, completion_status, and grade
    cursor.execute('''
        SELECT * FROM Enrollments
        WHERE payment_status IS NULL OR completion_status IS NULL OR grade IS NULL
    ''')
    missing_values = cursor.fetchall()

    if missing_values:
        print("Enrollments has missing values.")
        
    else:
        print("No missing values.")

# Test 3: Validate foreign key integrity for user_id
    cursor.execute('''
        SELECT e.enrollment_id, e.user_id
        FROM Enrollments e
        LEFT JOIN Users u ON e.user_id = u.user_id
        WHERE u.user_id IS NULL
    ''')
    invalid_users = cursor.fetchall()

    if invalid_users:
        print("Invalid user_id foreign keys in Enrollments:")
        for row in invalid_users:
            print(row)
    else:
        print("All user_id foreign keys are valid.")

    # Test 4: Validate foreign key integrity for course_id
    cursor.execute('''
        SELECT e.enrollment_id, e.course_id
        FROM Enrollments e
        LEFT JOIN Courses c ON e.course_id = c.course_id
        WHERE c.course_id IS NULL
    ''')
    invalid_courses = cursor.fetchall()

    if invalid_courses:
        print("Invalid course_id foreign keys in Enrollments:")
        for row in invalid_courses:
            print(row)
    else:
        print("All course_id foreign keys are valid.")

    # Test 5: check for invalid enrollment_type, user_id,course_id references in Enrollments table
    # Check for invalid enrollment_type (should be either 'Online' or 'Offline')
    cursor.execute('''
        SELECT * FROM Enrollments
        WHERE enrollment_type IS NULL OR (enrollment_type != 'Online' AND enrollment_type != 'Offline')
    ''')
    invalid_enrollment_type = cursor.fetchall()

    if invalid_enrollment_type:
        print("Enrollments with invalid enrollment_type:")
        for row in invalid_enrollment_type:
            print(row)
    else:
        print("All enrollment_type values are valid.")
        
    # Check for invalid user_id references in Enrollments
    cursor.execute('''
    SELECT e.enrollment_id, e.user_id
    FROM Enrollments e
    LEFT JOIN Users u ON e.user_id = u.user_id
    WHERE u.user_id IS NULL
    ''')
    invalid_users = cursor.fetchall()
    if invalid_users:
        print("Invalid user_id found in Enrollments table:")
        for row in invalid_users:
            print(row)
    else:
        print("All user_ids are valid in Enrollments.")
        
    # Check for invalid course_id references in Enrollments
    cursor.execute('''
    SELECT e.enrollment_id, e.course_id
    FROM Enrollments e
    LEFT JOIN Courses c ON e.course_id = c.course_id
    WHERE c.course_id IS NULL
    ''')
    invalid_courses = cursor.fetchall()
    if invalid_courses:
        print("Invalid course_id found in Enrollments table:")
        for row in invalid_courses:
            print(row)
    else:
        print("All course_ids are valid in Enrollments.")
    
    

# Close the connection
conn.close()


No duplicate enrollments.
Number of duplicate groups (based on payment_status, completion_status, and grade): 54
Enrollments has missing values.
All user_id foreign keys are valid.
All course_id foreign keys are valid.
All enrollment_type values are valid.
All user_ids are valid in Enrollments.
All course_ids are valid in Enrollments.
Total rows in Enrollments table: 2596


In [36]:
import sqlite3
import random
from faker import Faker

fake = Faker()

# Connect to the database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# Create the Reviews table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS Reviews (
    review_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    course_id INTEGER,
    review_date TEXT,
    rating INTEGER,
    comment TEXT,
    response TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
)
''')

# Clear existing data to avoid conflicts
cursor.execute("DELETE FROM Reviews")
conn.commit()

# Define possible values for reviews
ratings = [1, 2, 3, 4, 5, None]  # Ratings between 1 and 5, or None for missing data
responses = [None, "Thank you for your feedback!", "We appreciate your comments.", 
             "Your feedback is valuable!", "We'll work on it."]

# Define comment categories
positive_comments = [
    "The course was very informative and well-organized.",
    "I really enjoyed the interactive elements of this course.",
    "Great content and knowledgeable instructors!",
    "This course exceeded my expectations.",
    "Highly recommend for anyone interested in this subject."
]

neutral_comments = [
    "The course was okay but could use more examples.",
    "Decent content, but the pace was a bit slow.",
    "It was fine, but nothing exceptional.",
    "The course met the basic expectations.",
    "Not bad, but not great either."
]

negative_comments = [
    "The course lacked depth in the material.",
    "I found the instructor's explanations confusing.",
    "Not worth the price I paid for it.",
    "The content was outdated and not very engaging.",
    "I expected more practical examples."
]

# Function to generate a comment based on rating
def generate_comment(rating):
    if rating is None:
        return None  # Missing comment
    elif rating >= 4:
        return random.choice(positive_comments)
    elif rating == 3:
        return random.choice(neutral_comments)
    else:
        return random.choice(negative_comments)

# Generate unique random data for Reviews
review_data = []
unique_combinations = set()  # Track unique (user_id, course_id) combinations

for review_id in range(1, 1001):  # Generate 1000 reviews
    user_id = random.randint(1, 100)  # Random user from Users table
    course_id = random.randint(1, 100)  # Random course from Courses table

    # Ensure no duplicate (user_id, course_id) pairs
    while (user_id, course_id) in unique_combinations:
        user_id = random.randint(1, 100)
        course_id = random.randint(1, 100)

    unique_combinations.add((user_id, course_id))  # Track unique pairs

    review_date = fake.date_this_decade()  # Random date within the last decade
    rating = random.choice(ratings)  # Random rating
    comment = generate_comment(rating)  # Generate a realistic comment based on rating
    response = random.choice(responses)  # Random response

    # Append the review data
    review_data.append((review_id, user_id, course_id, review_date, rating, comment, response))

# Check that all user_ids and course_ids are valid (foreign keys)
valid_users = set()
cursor.execute("SELECT user_id FROM Users")
for row in cursor.fetchall():
    valid_users.add(row[0])

valid_courses = set()
cursor.execute("SELECT course_id FROM Courses")
for row in cursor.fetchall():
    valid_courses.add(row[0])

# Filter review data to only include valid user_id and course_id
filtered_review_data = [entry for entry in review_data if entry[1] in valid_users and entry[2] in valid_courses]

# Insert data into Reviews table
try:
    cursor.executemany('''
    INSERT INTO Reviews (
        review_id, user_id, course_id, review_date, rating, comment, response
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', filtered_review_data)

    # Commit the changes
    conn.commit()
    print("Reviews table created and randomized data inserted with realistic comments!")
except sqlite3.IntegrityError as e:
    print("IntegrityError occurred:", e)

# Query the first 10 rows from the Reviews table for verification
cursor.execute('SELECT * FROM Reviews LIMIT 10')
rows = cursor.fetchall()

# Print the retrieved rows
for row in rows:
    print(row)

# Close the connection
conn.close()


Reviews table created and randomized data inserted with realistic comments!
(1, 18, 24, '2020-08-14', 2, "I found the instructor's explanations confusing.", "We'll work on it.")
(2, 40, 32, '2021-01-06', 1, 'I expected more practical examples.', 'Thank you for your feedback!')
(3, 15, 71, '2022-04-13', None, None, 'Your feedback is valuable!')
(4, 4, 1, '2022-09-26', 5, 'This course exceeded my expectations.', "We'll work on it.")
(5, 54, 94, '2021-03-08', 4, 'I really enjoyed the interactive elements of this course.', 'We appreciate your comments.')
(6, 18, 22, '2022-05-21', None, None, 'Thank you for your feedback!')
(7, 84, 67, '2022-10-09', 2, 'Not worth the price I paid for it.', 'Thank you for your feedback!')
(8, 80, 10, '2022-03-01', 3, 'It was fine, but nothing exceptional.', "We'll work on it.")
(9, 9, 85, '2021-04-04', 3, 'Decent content, but the pace was a bit slow.', 'We appreciate your comments.')
(10, 42, 46, '2024-07-08', 3, 'Decent content, but the pace was a bit slow.

In [37]:
# Connect to the database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

#Step 1:Check for Missing Values (in rating, comment, and response columns)
cursor.execute('''
SELECT COUNT(*) FROM Reviews
WHERE rating IS NULL OR comment IS NULL OR response IS NULL
''')
missing_values_count = cursor.fetchone()[0]
print(f"Number of reviews with missing values: {missing_values_count}")

#Step 2:Check for Duplicate Reviews (if any user_id and course_id combinations appear more than once)
cursor.execute('''
SELECT user_id, course_id, COUNT(*)
FROM Reviews
GROUP BY user_id, course_id
HAVING COUNT(*) > 1
''')
duplicates = cursor.fetchall()
print(f"Duplicate user-course combinations: {duplicates}")

#Step 3:Data Integrity Check (validate foreign keys are correct)
# Check invalid user_ids
cursor.execute('''
SELECT review_id, user_id FROM Reviews
WHERE user_id NOT IN (SELECT user_id FROM Users)
''')
invalid_user_ids = cursor.fetchall()
print(f"Reviews with invalid user_ids: {invalid_user_ids}")

# Check invalid course_ids
cursor.execute('''
SELECT review_id, course_id FROM Reviews
WHERE course_id NOT IN (SELECT course_id FROM Courses)
''')
invalid_course_ids = cursor.fetchall()
print(f"Reviews with invalid course_ids: {invalid_course_ids}")

#Step 4:General Data Analysis
cursor.execute('''
SELECT AVG(rating), MIN(rating), MAX(rating) FROM Reviews
''')
rating_stats = cursor.fetchone()
print(f"Rating stats - AVG: {rating_stats[0]}, MIN: {rating_stats[1]}, MAX: {rating_stats[2]}")

# Close the connection
conn.close()


Number of reviews with missing values: 324
Duplicate user-course combinations: []
Reviews with invalid user_ids: []
Reviews with invalid course_ids: []
Rating stats - AVG: 2.926470588235294, MIN: 1, MAX: 5


In [61]:
import sqlite3
import random
from faker import Faker

fake = Faker()

# Connect to the database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

# Ensure Users and Enrollments tables have enough data for foreign keys
cursor.execute("SELECT COUNT(*) FROM Users")
user_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM Enrollments")
enrollment_count = cursor.fetchone()[0]

if user_count == 0 or enrollment_count == 0:
    print("Users or Enrollments tables do not have enough data. Please populate these tables first.")
else:
    # Create Payments table with constraints
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Payments (
        payment_id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        enrollment_id INTEGER NOT NULL,
        payment_date TEXT NOT NULL,
        amount REAL CHECK (amount >= 0),
        payment_method TEXT CHECK (payment_method IN ('Credit Card', 'PayPal', 'Bank Transfer')),
        transaction_id TEXT NOT NULL,
        payment_status TEXT CHECK (payment_status IN ('Paid', 'Pending', 'Failed')),
        FOREIGN KEY (user_id) REFERENCES Users(user_id),
        FOREIGN KEY (enrollment_id) REFERENCES Enrollments(enrollment_id)
    )
    ''')
    
    # Generate realistic payment data with missing values
    payment_data = []
    payment_methods = ['Credit Card', 'PayPal', 'Bank Transfer']
    payment_statuses = ['Paid', 'Pending', 'Failed']
    amounts = [round(random.uniform(2000, 5000),2)for _ in range(10000)]  # Random amount between 2000 and 50000 with 2 decimal points
    user_ids = [random.randint(1, user_count) for _ in range(1000)]  # Valid user IDs
    transaction_ids = [fake.uuid4()[:16] for _ in range(1000)]  # Shortened transaction ID (first 16 characters of UUID)
    payment_dates = [fake.date_this_decade() for _ in range(1000)]  # Random payment dates (always populated)

    # Fetch all valid enrollment_ids from the Enrollments table
    cursor.execute("SELECT enrollment_id FROM Enrollments")
    valid_enrollment_ids = [row[0] for row in cursor.fetchall()]


    # Adding optional missing data
    for i in range(1000):
        payment_date = payment_dates[i]  # Payment date is always filled
        amount = amounts[i] if random.random() > 0.05 else None  # 5% chance of missing amount
        payment_method = random.choice(payment_methods) if random.random() > 0.1 else None  # 10% chance of missing method
        transaction_id = transaction_ids[i]  # Transaction ID is always filled
        payment_status = random.choice(payment_statuses) if random.random() > 0.05 else None  # 5% chance of missing status
     
    # Ensure valid enrollment_id by picking a random enrollment_id from the existing ones
    enrollment_id = random.choice(valid_enrollment_ids)  # Randomly select a valid enrollment_id
    
    payment_data.append((
        user_ids[i],
        enrollment_id,  # Now using only a valid enrollment_id
        payment_date,
        amount,
        payment_method,
        transaction_id,
        payment_status
    ))


    # Insert the generated payment data into the Payments table
    try:
        cursor.executemany('''
        INSERT INTO Payments (user_id, enrollment_id, payment_date, amount, payment_method, transaction_id, payment_status)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', payment_data)

        # Commit the changes
        conn.commit()
        print("Payments table created and randomized data inserted with missing values.")
    except sqlite3.IntegrityError as e:
        print("IntegrityError occurred:", e)

# Query the first 10 rows from the Payments table for verification
cursor.execute('SELECT * FROM Payments LIMIT 10')
rows = cursor.fetchall()

# Print the retrieved rows
for row in rows:
    print(row)

# Close the connection
conn.close()


Payments table created and randomized data inserted with missing values.
(1, 529, 24939, '2020-08-23', 3860.17, 'PayPal', '89d58c06-6aba-4e', 'Pending')
(2, 1093, 711, '2021-03-20', 3129.06, 'Bank Transfer', 'e64ea2ce-82b5-4d', None)
(3, 1206, 1218, '2023-01-05', 4247.0, None, '3b948915-c806-40', 'Failed')
(4, 733, 1710, '2023-09-23', 2932.97, 'Bank Transfer', '1094fd08-4cb4-4b', 'Pending')
(5, 1239, 423910, '2020-07-09', 4674.26, 'Credit Card', '2a87f954-d07f-4b', 'Failed')
(6, 320, 1019, '2022-06-04', 4181.39, 'PayPal', 'f6a80111-7677-40', 'Failed')
(7, 786, 1601, '2023-08-18', 2198.79, None, '6bfd1de2-e948-44', 'Pending')
(8, 57, 17320, '2023-10-08', 4401.09, 'Credit Card', '7a5f762f-6388-4e', 'Failed')
(9, 274, 17145, '2022-06-11', 2649.43, 'Bank Transfer', 'fb9c1d3f-b96e-40', 'Paid')
(10, 330, 31122, '2021-10-02', 4246.59, 'PayPal', 'a88f3abc-8c39-46', 'Pending')


In [62]:
# Connect to the database
conn = sqlite3.connect('hybrid_learning@in.db')
cursor = conn.cursor()

#Step 1: Test for Missing Values
cursor.execute('''
SELECT COUNT(*) FROM Payments 
WHERE amount IS NULL OR payment_method IS NULL OR payment_status IS NULL
''')
missing_values_count = cursor.fetchone()[0]
print(f"Number of payments with missing values: {missing_values_count}")

#Step 2: Test for Duplicates
cursor.execute('''
SELECT user_id, enrollment_id, transaction_id, COUNT(*) 
FROM Payments 
GROUP BY user_id, enrollment_id, transaction_id
HAVING COUNT(*) > 1
''')
duplicates = cursor.fetchall()
if duplicates:
    print("Duplicate records found:")
    for duplicate in duplicates:
        print(duplicate)
else:
    print("No duplicates found.")

#Step 3: Test for Invalid foreign Keys

 # checking if any invalid user_ids
cursor.execute('''
SELECT COUNT(*) FROM Payments 
WHERE user_id NOT IN (SELECT user_id FROM Users)
''')
invalid_user_ids = cursor.fetchone()[0]
print(f"Number of payments with invalid user_ids: {invalid_user_ids}")

# checking if any invalid enrollment_ids
cursor.execute('''
SELECT COUNT(*) FROM Payments 
WHERE enrollment_id NOT IN (SELECT enrollment_id FROM Enrollments)
''')
invalid_enrollment_ids = cursor.fetchone()[0]
print(f"Number of payments with invalid enrollment_ids: {invalid_enrollment_ids}")

# checking if any invalid amounts
cursor.execute('''
SELECT COUNT(*) FROM Payments 
WHERE amount < 0
''')
invalid_amounts = cursor.fetchone()[0]
print(f"Number of payments with invalid amount: {invalid_amounts}")

# checking if any invalid payment_methods
cursor.execute('''
SELECT COUNT(*) FROM Payments 
WHERE payment_method NOT IN ('Credit Card', 'PayPal', 'Bank Transfer')
''')
invalid_payment_methods = cursor.fetchone()[0]
print(f"Number of payments with invalid payment_method: {invalid_payment_methods}")

#checking if any invalid payment_status
cursor.execute('''
SELECT COUNT(*) FROM Payments 
WHERE payment_status NOT IN ('Paid', 'Pending', 'Failed')
''')
invalid_payment_status = cursor.fetchone()[0]
print(f"Number of payments with invalid payment_status: {invalid_payment_status}")


Number of payments with missing values: 3
No duplicates found.
Number of payments with invalid user_ids: 0
Number of payments with invalid enrollment_ids: 0
Number of payments with invalid amount: 0
Number of payments with invalid payment_method: 0
Number of payments with invalid payment_status: 0
